In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import utils
import json
import os
import git
import warnings
warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY

## Get the necessary data from Hopsworks

In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
# secrets = utils.secrets_api(project.name)

CITY = "dublin"
STATION = "HEUSTON BRIDGE (NORTH)"

# latitude =
# longitude =

today = datetime.datetime.now()

2025-01-28 07:52:35,664 INFO: Initializing external client


2025-01-28 07:52:35,666 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-01-28 07:52:37,218 INFO: Python Engine initialized.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494


In [4]:
# Retrieve feature groups
bike_fg = fs.get_feature_group(
    name='bike_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)


## Clone and pull the repository with the bike data

In [5]:
# Configuration
REPO_URL = "https://github.com/MaxHalford/bike-sharing-history"
CLONE_DIR = "./bike_data/bike-sharing-history"
TARGET_CITY = "dublin"
FILE_NAME = "jcdecaux.geojson"

In [6]:
if not os.path.exists(CLONE_DIR):
    print("Cloning repository...")
    git.Repo.clone_from(REPO_URL, CLONE_DIR)
repo = git.Repo(CLONE_DIR)

# go to main and pull the latest changes
repo.git.checkout("main", force=True)
repo.remotes.origin.pull()

## Get the latest datetime present in the bike data

In [7]:
# last_bike_datetime = "2025-01-06 15:06:11 UTC"

bike_df = bike_fg.read()

last_bike_datetime = bike_df["datetime"].max()
last_bike_datetime = last_bike_datetime.strftime("%Y-%m-%d %H:%M:%S %Z")

last_bike_datetime

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.11s) 


'2025-01-28 00:00:00 UTC'

## Loop through the commits and convert the bike data into a dataframe

In [8]:
results = {}

# Populate the results dict with the stations
data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
if os.path.exists(data_file):
    with open(data_file, "r") as f:
        data = f.read()
        data = json.loads(data)
        for feature in data["features"]:
            name = feature["properties"]["name"]
            results[name] = []

In [9]:
start_time = datetime.datetime.now()
last_day_and_hour = None

for commit in repo.iter_commits():
    # Stop when we reach the earliest bike date
    if commit.committed_datetime <= datetime.datetime.strptime(last_bike_datetime, "%Y-%m-%d %H:%M:%S %Z").replace(tzinfo=datetime.timezone.utc):
        print("breaking at: ", commit.committed_datetime)
        break

    # Skip commits from today
    if commit.committed_datetime > today.replace(tzinfo=datetime.timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0):
        first_commited_datetime = repo.commit().committed_datetime
        continue

    day_and_hour = commit.committed_datetime.replace(minute=0, second=0, microsecond=0)
    # print("day_and_hour: ", day_and_hour, " - last_day_and_hour: ", last_day_and_hour)
    if day_and_hour == last_day_and_hour:
        continue
    last_day_and_hour = day_and_hour

    # Get the data for the commit
    print("Processing commit: ", commit.hexsha, " - ", commit.committed_datetime)
    try:
        repo.git.checkout(commit.hexsha, force=True)
    except Exception as e:
        print("Error checking out commit: ", e)
        break

    data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
    if os.path.exists(data_file):
        with open(data_file, "r") as f:
            data = f.read()
            data = json.loads(data)
            for feature in data["features"]:
                try:
                    results[feature["properties"]["name"]].append([feature["properties"]["available_bikes"], commit.committed_datetime])
                except KeyError:
                    continue

print(results)


breaking at:  2025-01-27 23:49:07+00:00
{'CLARENDON ROW': [], 'BLESSINGTON STREET': [], 'BOLTON STREET': [], 'GREEK STREET': [], 'CHARLEMONT PLACE': [], 'CHRISTCHURCH PLACE': [], 'HIGH STREET': [], 'CUSTOM HOUSE QUAY': [], 'EXCHEQUER STREET': [], 'DAME STREET': [], 'EARLSFORT TERRACE': [], 'ECCLES STREET': [], 'FITZWILLIAM SQUARE WEST': [], 'FOWNES STREET UPPER': [], 'HARDWICKE STREET': [], 'GEORGES QUAY': [], 'GOLDEN LANE': [], 'GRANTHAM STREET': [], 'HERBERT PLACE': [], 'JAMES STREET EAST': [], 'LEINSTER STREET SOUTH': [], 'TOWNSEND STREET': [], 'CUSTOM HOUSE': [], 'CATHAL BRUGHA STREET': [], 'MERRION SQUARE EAST': [], 'MERRION SQUARE WEST': [], 'MOLESWORTH STREET': [], 'MOUNTJOY SQUARE WEST': [], 'ORMOND QUAY UPPER': [], 'PARNELL SQUARE NORTH': [], 'PARNELL STREET': [], 'PEARSE STREET': [], "PRINCES STREET / O'CONNELL STREET": [], 'PORTOBELLO HARBOUR': [], 'SMITHFIELD': [], "ST. STEPHEN'S GREEN EAST": [], "ST. STEPHEN'S GREEN SOUTH": [], 'TALBOT STREET': [], 'WILTON TERRACE': [], 'J

In [10]:
# turn results into a dataframe
df_bike_today = pd.DataFrame()

for station, values in results.items():
    if len(values) > 0:
        df = pd.DataFrame(values, columns=["num_bikes_available", "datetime"])
        df["datetime"] = pd.to_datetime(df["datetime"], utc=True)
        df["station"] = station.replace(" ", "_")
        df_bike_today = pd.concat([df_bike_today, df])

# if empty, do nothing
if not df_bike_today.empty:
    df_bike_today.dropna(inplace=True)
    df_bike_today["num_bikes_available"] = df_bike_today["num_bikes_available"].astype("float32")
    df_bike_today = df_bike_today[df_bike_today['station'].isin([STATION.replace(" ", "_")])]
    df_bike_today['datetime'] = pd.to_datetime(df_bike_today['datetime'], utc=True).dt.floor('H') + datetime.timedelta(hours=1)
df_bike_today


""


## Fetch the weather data for the same time period

In [11]:
forecast_df = utils.get_hourly_weather_forecast(CITY)
forecast_df = forecast_df.rename(columns={'date_x': 'datetime'})
forecast_df = forecast_df.drop(columns=['date_y', 'date_only'])
forecast_df.dropna(inplace=True)

print(forecast_df.empty)

forecast_df

features: {'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
params: {'latitude': 53.35, 'longitude': -6.26, 'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
Coordinates 53.5°N -6.25°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
False


,datetime,temperature_2m,apparent_temperature,rain,snowfall,wind_speed_10m,daylight_duration,rain_sum,city
0,2025-01-28 00:00:00+00:00,5.60,4.084646,0.0,0.0,2.276840,31429.511719,4.8,dublin
1,2025-01-28 01:00:00+00:00,5.40,3.280367,0.0,0.0,6.130579,31429.511719,4.8,dublin
2,2025-01-28 02:00:00+00:00,5.30,2.636153,0.0,0.0,9.931042,31429.511719,4.8,dublin
3,2025-01-28 03:00:00+00:00,5.35,2.213164,0.0,0.0,13.246826,31429.511719,4.8,dublin
4,2025-01-28 04:00:00+00:00,5.75,2.258863,1.1,0.0,16.263872,31429.511719,4.8,dublin
...,...,...,...,...,...,...,...,...,...
235,2025-02-06 19:00:00+00:00,7.85,3.175895,0.0,0.0,20.873791,33421.378906,0.0,dublin
236,2025-02-06 20:00:00+00:00,7.50,2.722842,0.0,0.0,21.650938,33421.378906,0.0,dublin
237,2025-02-06 21:00:00+00:00,7.15,2.268211,0.0,0.0,22.183128,33421.378906,0.0,dublin
238,2025-02-06 22:00:00+00:00,6.75,1.752871,0.0,0.0,22.725668,33421.378906,0.0,dublin


## Insert the bike and weather data into Hopsworks

In [12]:
if df_bike_today.empty:
    print("No bike data available for today")
else:
    bike_fg.insert(df_bike_today)

No bike data available for today


In [13]:
if forecast_df.empty:
    print("No weather forecast available for today")
else:
    weather_fg.insert(forecast_df, write_options={"wait_for_job": True})


Uploading Dataframe: 0.00% |                                     | Rows 0/240 | Elapsed Time: 00:00 | Remaining Time: ?

Uploading Dataframe: 100.00% |█████████████████████████████| Rows 240/240 | Elapsed Time: 00:00 | Remaining Time: 00:00

Launching job: weather_data_1_offline_fg_materialization


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/weather_data_1_offline_fg_materialization/executions


2025-01-28 07:53:00,686 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED


2025-01-28 07:53:03,895 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED


2025-01-28 07:53:07,099 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED


2025-01-28 07:55:39,038 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED


2025-01-28 07:55:39,205 INFO: Waiting for log aggregation to finish.


2025-01-28 07:56:08,149 INFO: Execution finished successfully.
